In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# IWB temp data from online source
weatherbuoy_df = pd.read_csv('https://erddap.marine.ie/erddap/tabledap/IWBNetwork.csvp?station_id%2Ctime%2CAtmosphericPressure%2CWindDirection%2CWindSpeed%2CWaveHeight%2CWavePeriod%2CSeaTemperature%2Csalinity')
weatherbuoy_df['time (UTC)'] = pd.to_datetime(weatherbuoy_df['time (UTC)'])

# Filter for just 1 weather buoy
# m5 = Celtic Sea
m5_df = weatherbuoy_df[(weatherbuoy_df.station_id == 'M5')]

In [ ]:
# Define the wind directions
bins_dir = [0, 11.25, 33.75, 56.25, 78.75,101.25,123.75,146.25,168.75,191.25,213.75,236.25,258.75,281.25,303.75,326.25,348.75, 360.00]
bins_dir_labels = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW','North']

# categorise wind speed
m5_df['Wind Speed Category (knots)'] = pd.cut(m5_df['WindSpeed (knots)'], 
                                              bins=[0, 4, 8, 12, 16, 20, 24, 1000], 
                                              include_lowest=True, 
                                              labels=['4-8', '8-12', '12-16', '16-20', '20-24', '24-28', '28+'])
m5_df['dir_binned'] = pd.cut(m5_df['WindDirection (degrees true)'], bins_dir, labels=bins_dir_labels)

# new df for only necessary columns
df_simplfied = m5_df[['Wind Speed Category (knots)', 'dir_binned']].copy()

# Group Wind Speed and Direction and calculate frequency of them occurring
df_grouped = df_simplfied.groupby(['Wind Speed Category (knots)','dir_binned']).size().reset_index(name="frequency") # grouping
df_grouped.reset_index(inplace=True) 
df_grouped['percentage'] = df_grouped['frequency']/df_grouped['frequency'].sum()
df_grouped['percentage%'] = df_grouped['percentage']*100
df_grouped = df_grouped.replace(r'North', 'N', regex=True)

fig = px.bar_polar(df_grouped, r="percentage%", theta="dir_binned",
                   color="Wind Speed Category (knots)", template="plotly_dark",
                   color_discrete_sequence= px.colors.sequential.Plasma_r,
                  title='Wind Speed and Direction in the Celtic Sea using M5 weather buoy')
fig.update_layout(polar_radialaxis_showticklabels=False)
fig.show()
fig.write_image('../content/day15_wind_roseplot.png')